## Demoing `defog_utils`

**Purpose**
- Be able to switch between different LLM providers without having to rewrite the code

**Assumptions**
- Every LLM provider will have common features, like structure outputs, and streaming. There are features that are specific to individual LLM providers (like predicted outputs for OpenAI models). But for the vast majority of use-cases, we can use chat_async

In [1]:
import os
os.environ["DEEPSEEK_API_KEY"] = "sk-6a95286f3a4243d4ba209d76db36595e"
from defog_utils.utils_multi_llm import chat_async


# remember to have the following in your environment:
# - OPENAI_API_KEY
# - GEMINI_API_KEY
# - ANTHROPIC_API_KEY
# - DEEPSEEK_API_KEY
# (optionally) - TOGETHER_API_KEY

In [18]:
from pprint import pprint

def pretty_print_llm_response(resp):
    if resp.output_tokens_details:
        resp.output_tokens_details = resp.output_tokens_details.__dict__
    
    if type(resp.content) != str:
        resp.content = resp.content.__dict__

    # format cost_in_cents to 3 significant figures
    # it's okay to use scientific notation for this
    sig_figs = 3
    format_string = f"{{:.{sig_figs - 1}e}}"
    resp.cost_in_cents = format_string.format(resp.cost_in_cents)
    pprint(resp.__dict__, width=100, indent=2)

### First, let's do normal chat messages

In [3]:
messages = [
    {
        "role": "system",
        "content": "Your task is to generate SQL given a natural language question and schema of the user's database. Do not use aliases. Return only the SQL without ```."
    },
    {
        "role": "user",
        "content": f"""Question: What is the total number of orders?
Schema:
```sql
CREATE TABLE orders (
    order_id int,
    customer_id int,
    employee_id int,
    order_date date
);
```"""
}]

#### Let's try gpt-4o-mini first

In [4]:
resp = await chat_async(
    model="gpt-4o-mini",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
pretty_print_llm_response(resp)

{   'content': 'SELECT COUNT(*) FROM '
               'orders;',
    'cost_in_cents': '3.72e-04',
    'input_tokens': 82,
    'model': 'gpt-4o-mini',
    'output_tokens': 6,
    'output_tokens_details': {   'accepted_prediction_tokens': 0,
                                 'audio_tokens': 0,
                                 'reasoning_tokens': 0,
                                 'rejected_prediction_tokens': 0},
    'time': 0.639}


#### Now let's try gpt-4o

In [5]:
resp = await chat_async(
    model="gpt-4o",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
pretty_print_llm_response(resp)

{   'content': 'SELECT COUNT(*) FROM '
               'orders;',
    'cost_in_cents': '6.21e-03',
    'input_tokens': 82,
    'model': 'gpt-4o',
    'output_tokens': 6,
    'output_tokens_details': {   'accepted_prediction_tokens': 0,
                                 'audio_tokens': 0,
                                 'reasoning_tokens': 0,
                                 'rejected_prediction_tokens': 0},
    'time': 0.652}


#### Now let's try o1-mini

Note that o1 models do not support temperature OR system prompts. defog_utils will automatically take care of this

In [6]:
resp = await chat_async(
    model="o1-mini",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
pretty_print_llm_response(resp)

{   'content': 'SELECT COUNT(*) FROM '
               'orders;',
    'cost_in_cents': '1.79e-01',
    'input_tokens': 84,
    'model': 'o1-mini',
    'output_tokens': 149,
    'output_tokens_details': {   'accepted_prediction_tokens': 0,
                                 'audio_tokens': 0,
                                 'reasoning_tokens': 128,
                                 'rejected_prediction_tokens': 0},
    'time': 2.041}


#### Finally, try o1

In [7]:
resp = await chat_async(
    model="o1",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
pretty_print_llm_response(resp)

{   'content': 'SELECT COUNT(*) FROM '
               'orders;',
    'cost_in_cents': '9.02e-01',
    'input_tokens': 107,
    'model': 'o1',
    'output_tokens': 150,
    'output_tokens_details': {   'accepted_prediction_tokens': 0,
                                 'audio_tokens': 0,
                                 'reasoning_tokens': 128,
                                 'rejected_prediction_tokens': 0},
    'time': 2.582}


### We can use exactly the same thing with non-openai models as well

In [10]:
# claude-3.5-sonnet
resp = await chat_async(
    model="claude-3-5-sonnet-latest",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
print("CLAUDE 3.5 SONNET")
pretty_print_llm_response(resp)

# gemini-2.0-flash-exp
resp = await chat_async(
    model="gemini-2.0-flash-exp",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
print("GEMINI 2.0 FLASH EXP")
pretty_print_llm_response(resp)

# deepseek-reasoner
resp = await chat_async(
    model="deepseek-reasoner",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
)
print("DEEPSEEK REASONER")
pretty_print_llm_response(resp)

CLAUDE 3.5 SONNET
{   'content': 'SELECT COUNT(*) FROM '
               'orders',
    'cost_in_cents': '1.42e-02',
    'input_tokens': 245,
    'model': 'claude-3-5-sonnet-latest',
    'output_tokens': 9,
    'output_tokens_details': None,
    'time': 0.644}
GEMINI 2.0 FLASH EXP
{   'content': '```sql\n'
               'SELECT count(*) FROM '
               'orders\n'
               '```',
    'cost_in_cents': '3.18e-04',
    'input_tokens': 246,
    'model': 'gemini-2.0-flash-exp',
    'output_tokens': 10,
    'output_tokens_details': None,
    'time': 0.614}
DEEPSEEK REASONER
{   'content': 'SELECT COUNT(*) FROM '
               'orders;',
    'cost_in_cents': '5.55e-02',
    'input_tokens': 238,
    'model': 'deepseek-reasoner',
    'output_tokens': 253,
    'output_tokens_details': {   'accepted_prediction_tokens': None,
                                 'audio_tokens': None,
                                 'reasoning_tokens': 245,
                                 'rejected_predict

### Now, let's try the same thing but with structured outputs

In [19]:
from pydantic import BaseModel

class ResponseFormat(BaseModel):
    reasoning: str
    sql: str

resp = await chat_async(
    model="gpt-4o",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
    response_format=ResponseFormat,
)
print("GPT4o")
pretty_print_llm_response(resp)

GPT4o
{ 'content': { 'reasoning': 'To find the total number of orders, we need to count the number of '
                            "rows in the 'orders' table. The SQL COUNT function is used to count "
                            'the number of rows in a table. Since we want the total number of '
                            "orders, we can use COUNT(*) to count all rows in the 'orders' table.",
               'sql': 'SELECT COUNT(*) FROM orders;'},
  'cost_in_cents': '7.88e-02',
  'input_tokens': 310,
  'model': 'gpt-4o',
  'output_tokens': 78,
  'output_tokens_details': { 'accepted_prediction_tokens': 0,
                             'audio_tokens': 0,
                             'reasoning_tokens': 0,
                             'rejected_prediction_tokens': 0},
  'time': 2.486}


In [20]:
from pydantic import BaseModel

class ResponseFormat(BaseModel):
    reasoning: str
    sql: str

resp = await chat_async(
    model="gemini-2.0-flash-exp",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
    response_format=ResponseFormat,
)
print("Gemini 2.0 Flash")
pretty_print_llm_response(resp)

Gemini 2.0 Flash
{ 'content': { 'reasoning': 'To find the total number of orders, I need to count the number of '
                            "rows in the 'orders' table.",
               'sql': 'SELECT count(*) FROM orders'},
  'cost_in_cents': '1.31e-03',
  'input_tokens': 278,
  'model': 'gemini-2.0-flash-exp',
  'output_tokens': 43,
  'output_tokens_details': None,
  'time': 1.918}


In [21]:
from pydantic import BaseModel

class ResponseFormat(BaseModel):
    reasoning: str
    sql: str

resp = await chat_async(
    model="o1",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
    response_format=ResponseFormat,
)
print("Gemini 2.0 Flash")
pretty_print_llm_response(resp)

Gemini 2.0 Flash
{ 'content': { 'reasoning': 'We need to find the total count of all orders in the orders table, so '
                            'we use COUNT(*) on that table.',
               'sql': 'SELECT COUNT(*) FROM orders'},
  'cost_in_cents': '2.58e+00',
  'input_tokens': 305,
  'model': 'o1',
  'output_tokens': 430,
  'output_tokens_details': { 'accepted_prediction_tokens': 0,
                             'audio_tokens': 0,
                             'reasoning_tokens': 384,
                             'rejected_prediction_tokens': 0},
  'time': 12.532}


In [23]:
# if you try a model that does not support response_format
# it will just return the content as a string

from pydantic import BaseModel

class ResponseFormat(BaseModel):
    reasoning: str
    sql: str

resp = await chat_async(
    model="o1-mini",
    messages=messages,
    max_completion_tokens=4000,
    temperature=0.0,
    seed=0,
    response_format=ResponseFormat,
)
print("o1-mini")
pretty_print_llm_response(resp)

o1-mini
{ 'content': 'SELECT COUNT(*) FROM orders;',
  'cost_in_cents': '2.14e-04',
  'input_tokens': 332,
  'model': 'deepseek-chat',
  'output_tokens': 6,
  'output_tokens_details': None,
  'time': 1.644}
